In [62]:
from dewloosh.geom import PolyData
from dewloosh.geom.tri import triangulate
from dewloosh.geom.space import StandardFrame
from dewloosh.geom.utils import centralize
from dewloosh.math.linalg import linspace
from dewloosh.geom.tri import triangulate
from dewloosh.geom import grid
from dewloosh.geom.tri.triutils import get_points_inside_triangles
from sectionproperties.pre.library.steel_sections import i_section as ISection
from sectionproperties.pre.library.steel_sections import circular_hollow_section as CHS
import numpy as np

In [63]:
points = []

In [64]:
# section
A = StandardFrame(dim=3)
geometry = ISection(d=203, b=133, t_f=7.8, t_w=5.8, r=8.9, n_r=16)
#geometry = CHS(d=200, t=10, n=64)
mesh = geometry.create_mesh(mesh_sizes=[3.0])
coords = centralize(np.array(geometry.mesh['vertices']))
topo = np.array(geometry.mesh['triangles'].tolist())[:, :3]
section = PolyData(coords=coords, topo=topo, frame=A)
points.append(section.coords())

In [65]:
section.plot()

Triangulate a single section embedded in a box

In [66]:

padding = 0.2
points_per_edge = 100

dx = coords[:, 0].max() - coords[:, 0].min()
dy = coords[:, 1].max() - coords[:, 1].min()
Lxy = (1 + padding) * max(dx, dy)
Lx, Ly, Lz = Lxy, Lxy, Lxy

# grid points inside face
N = points_per_edge + 2
Grid = grid(size=(Lx*0.99, Ly*0.99), shape=(N, N), eshape='Q4', centralize=True)
centers = PolyData(coords=Grid[0], topo=Grid[1], frame=A).centers()
ids = get_points_inside_triangles(coords, topo, centers).astype(bool)
points.append(centers[~ids])

# grid points on edges
xmin, xmax = -Lx/2, Lx/2
ymin, ymax = -Ly/2, Ly/2
points.append(linspace(np.array([xmin, ymin, 0]), 
                       np.array([xmax, ymin, 0]), N)[1:-1])
points.append(linspace(np.array([xmax, ymin, 0]), 
                       np.array([xmax, ymax, 0]), N)[1:-1])
points.append(linspace(np.array([xmax, ymax, 0]), 
                       np.array([xmin, ymax, 0]), N)[1:-1])
points.append(linspace(np.array([xmin, ymax, 0]), 
                       np.array([xmin, ymin, 0]), N)[1:-1])

# grid points on corners
points.append(np.array([[xmin, ymin, 0]]))
points.append(np.array([[xmax, ymin, 0]]))
points.append(np.array([[xmax, ymax, 0]]))
points.append(np.array([[xmin, ymax, 0]]))

# put it all together
points = np.vstack(points)

In [67]:
# triangulate nad plot
coords, topo, _ = triangulate(points=points)
section = PolyData(coords=coords, topo=topo, frame=A)
section.centralize()
section.plot()